lunana  
last update 2022 04 26  
ゆっくりしていってね！

version 6 +Trends

# Trends  
Ubiquantコンペは、Optiverコンペに似ています。  
https://www.kaggle.com/c/optiver-realized-volatility-prediction  
training timelineとforcasting timelineで訓練データが変わるかもしれない。  
notebookが実行されるごとに、モデル訓練が必要かもしれません。  
そこで、効果検証用のnotebookと最終提出用のnotebook２つを作っていきます。  

The Ubiquant competition is similar to the Optiver competition.  
https://www.kaggle.com/c/optiver-realized-volatility-prediction  
Training data may change between training timeline and forcasting timeline.  
Model training may be required each time the notebook is run.  
Therefore, we will create two notebooks, one for effect verification and the other for final submission.  

## EDA  
https://www.kaggle.com/lunapandachan/ubiquant-eda-english

## Add test  
* 🔥Ubiquant DNN chiranjeev  
version 5　seed=41, 5folds,LB=0.148  
version 7　seed=41, 10folds, LB=0.149  
version 8　seed=41, 10folds, LB=0.149  


# Score  
version 1　val_correlation LB=0.1522  
version 2　val_mae LB=0.1496  
version 3　val_loss LB=0.1481  
version 4　val_mape LB=0.1495  
version 5　val_rmse LB=0.1492  

検証した結果、version1のval_correlationが一番高いスコアでした。

**霊夢:今日はLonnieさんのコードを追試するよ。**

**Reimu: Today I'll retest Lonnie's code.**  

https://www.kaggle.com/lonnieqin/ubiquant-market-prediction-with-dnn

現時点では、supplemental_train.csvを使用すると、Submission Scoring Errorが出ます。これは、ノイズデータのためと考えます。4月18日以降は正しいsuplemental_train.csvになるようなので、これに期待して、スイッチをつけることにしました。 4月17日以降に提出するときには、注意してください。

In [ ]:
import datetime
dt_now = datetime.datetime.now()
IS_SUP=False
if dt_now.month>5 or dt_now.day>16:
    IS_SUP=True

# CFG

In [ ]:
class CFG:
    n_folds=5
    train_filename='../input/ubiquant-market-prediction-half-precision-pickle/train.pkl'
    s_filename='../input/ubiquant-market-prediction/supplemental_train.csv'
    train_dataset_path='../input/ubiquant-tfrecords/'

## import

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import warnings

import time
import pickle
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K

In [ ]:
%%time

n_features = 300
features = [f'f_{i}' for i in range(n_features)]
dtype_features={'time_id':int,'investment_id':int,'target':np.float32}
usecols=['time_id','investment_id','target']
for feature in features:
    dtype_features[feature]= np.float32
    usecols.append(feature)

In [ ]:
if IS_SUP:
    f=open(CFG.s_filename,'r')
    strain_file=open('s_train.csv', 'w')
    flg_first=True
    index_f=None
    count =0
    rows=[]
    p_count=0
    for row in f:
        if flg_first:
            index_f=row
            strain_file.write(index_f)
            flg_first=False
        else:
            if count<200000:
                count+=1
                strain_file.write(row)
            else:
                count=0
                strain_file.write(row)
                strain_file.close()
                ## procedure s_train to df
                s_train=pd.read_csv(f's_train.csv',usecols=usecols,dtype=dtype_features)
                s_train.to_parquet(f's_train.parquet{p_count}')
                p_count+=1                   
                !rm s_train.csv
                del s_train
                gc.collect()
                ! free -m
                strain_file=open('s_train.csv', 'w')
                strain_file.write(index_f)            
    f.close()
    strain_file.close()
    df=pd.read_csv(f's_train.csv',usecols=usecols,dtype=dtype_features)
    !rm s_train.csv
    for idx in range(p_count):
        s_train = pd.read_parquet(f's_train.parquet{idx}')
        df=pd.concat([df,s_train])
        del s_train
        !rm s_train.parquet{idx}
    gc.collect()
    !free -m
    df = df.reset_index(drop=True)
    df.to_pickle(f's_train.pkl')

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import numpy as np
from scipy.special import comb
from itertools import combinations

class CombinatorialPurgedGroupKFold():
    def __init__(self, n_splits = 6, n_test_splits = 2, purge = 1, pctEmbargo = 0.01, **kwargs):
        self.n_splits = n_splits
        self.n_test_splits = n_test_splits
        self.purge = purge
        self.pctEmbargo = pctEmbargo
        
    def split(self, X, y = None, groups = None):
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
            
        u, ind = np.unique(groups, return_index = True)
        unique_groups = u[np.argsort(ind)]
        n_groups = len(unique_groups)
        group_dict = {}
        for idx in range(len(X)):
            if groups[idx] in group_dict:
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
                
        n_folds = comb(self.n_splits, self.n_test_splits, exact = True)
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
            
        mbrg = int(n_groups * self.pctEmbargo)
        if mbrg < 0:
            raise ValueError(
                "The number of 'embargoed' groups should not be negative")
        
        split_dict = {}
        group_test_size = n_groups // self.n_splits
        for split in range(self.n_splits):
            if split == self.n_splits - 1:
                split_dict[split] = unique_groups[int(split * group_test_size):].tolist()
            else:
                split_dict[split] = unique_groups[int(split * group_test_size):int((split + 1) * group_test_size)].tolist()
        
        for test_splits in combinations(range(self.n_splits), self.n_test_splits):
            test_groups = []
            banned_groups = []
            for split in test_splits:
                test_groups += split_dict[split]
                banned_groups += unique_groups[split_dict[split][0] - self.purge:split_dict[split][0]].tolist()
                banned_groups += unique_groups[split_dict[split][-1] + 1:split_dict[split][-1] + self.purge + mbrg + 1].tolist()
            train_groups = [i for i in unique_groups if (i not in banned_groups) and (i not in test_groups)]

            train_idx = []
            test_idx = []
            for train_group in train_groups:
                train_idx += group_dict[train_group]
            for test_group in test_groups:
                test_idx += group_dict[test_group]
            yield train_idx, test_idx

In [ ]:
if IS_SUP:
    investment_id = df.pop("investment_id")
    time_id = df.pop("time_id")
    investment_id.head()

In [ ]:
if IS_SUP:
    y = df.pop("target")
    y.head()

In [ ]:
def create_record(i):
    dic = {}
    dic[f"features"] = tf.train.Feature(float_list=tf.train.FloatList(value=list(df.iloc[i])))
    dic["time_id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[time_id.iloc[i]]))
    dic["investment_id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[investment_id.iloc[i]]))
    dic["target"] = tf.train.Feature(float_list=tf.train.FloatList(value=[y.iloc[i]]))
    record_bytes = tf.train.Example(features=tf.train.Features(feature=dic)).SerializeToString()
    return record_bytes
    
def decode_function(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {
          "features": tf.io.FixedLenFeature([300], dtype=tf.float32),
          "time_id": tf.io.FixedLenFeature([], dtype=tf.int64),
          "investment_id": tf.io.FixedLenFeature([], dtype=tf.int64),
          "target": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )

In [ ]:
%%time
if IS_SUP:
    import time
    n_splits = CFG.n_folds
    n_test_splits = 1
    kfold = CombinatorialPurgedGroupKFold(n_splits, n_test_splits)
    for fold, (train_indices, test_indices) in enumerate(kfold.split(df, groups=time_id)):
        print("=" * 100)
        print(f"Fold {fold}")
        print("=" * 100)
        print("Train Sample size:", len(test_indices))
        train_save_path = f"s_train_fold{fold}.tfrecords"
        begin = time.time()
        print(f"Creating {train_save_path}")
        with tf.io.TFRecordWriter(train_save_path) as file_writer:
            for i in test_indices:
                file_writer.write(create_record(i))
        print("Elapsed time: %.2f"%(time.time() - begin))
    del df,time_id,y
    gc.collect()

In [ ]:
!free -m

# Model DNN

In [ ]:
train = pd.read_pickle(CFG.train_filename)
if IS_SUP:
    investment_id_2 = train.pop("investment_id")
    investment_id=pd.concat([investment_id,investment_id_2])
else:
    investment_id = train.pop("investment_id")    
del train
gc.collect()
investment_id.head()

In [ ]:
%%time
investment_id=investment_id.astype(np.int16)
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
with tf.device("cpu"):
    investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
    investment_id_lookup_layer.adapt(pd.DataFrame({"investment_ids":investment_ids}))

## Make Tensorflow dataset

In [ ]:
def decode_function(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {
          "features": tf.io.FixedLenFeature([300], dtype=tf.float32),
          "time_id": tf.io.FixedLenFeature([], dtype=tf.int64),
          "investment_id": tf.io.FixedLenFeature([], dtype=tf.int64),
          "target": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )
def preprocess(item):
    return (item["investment_id"], item["features"]), item["target"]
def make_dataset(file_paths, batch_size=4096, mode="train"):
    ds = tf.data.TFRecordDataset(file_paths)
    ds = ds.map(decode_function)
    ds = ds.map(preprocess)
    #if mode == "train":
     #   ds = ds.shuffle(batch_size * 4)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

## Modeling

In [ ]:
def correlation(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr

def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.1)(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.1)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
    return model

Let's take a look at this Model's architecture.

In [ ]:
model = get_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

## Model Training

**魔理沙:ここからがtrainingだぜ。**  
**Marisa: Training starts here.**

In [ ]:
import time

In [ ]:
def train_fold(index):
    train_path = []
    valid_path = []
    for i in range(CFG.n_folds):
        if i==index:
            valid_path.append(f"{CFG.train_dataset_path}train_fold{i}.tfrecords")
            if IS_SUP:
                valid_path.append(f"s_train_fold{i}.tfrecords")
        else:
            train_path.append(f"{CFG.train_dataset_path}train_fold{i}.tfrecords")
            if IS_SUP:
                train_path.append(f"s_train_fold{i}.tfrecords")

    train_ds = make_dataset(train_path)
    valid_ds = make_dataset(valid_path, mode="valid")
    print(f' fold{index}  ')
    model = get_model()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}.tf", monitor="val_correlation", mode="min", save_best_only=True, save_weights_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model.load_weights(f"model_{index}.tf")
    model.save(f'model_{index}')
    #for metric in ["loss", "mae", "mape", "rmse", "correlation"]:
    #    pd.DataFrame(history.history, columns=[metric, f"val_{metric}"]).plot()
    #    plt.title(metric.upper())
    #    plt.show()
    #y_vals = []
    #for _, y in valid_ds:
    #    y_vals += list(y.numpy().reshape(-1))
    #y_val = np.array(y_vals)
    #pearson_score = stats.pearsonr(model.predict(valid_ds).reshape(-1), y_val)[0]
    #print(f"Pearson Score: {pearson_score}　fold{index} ")
    del model,train_ds,valid_ds
    K.clear_session()
    gc.collect()
    #time.sleep(10)
    !free -m

In [ ]:
for index in range(CFG.n_folds):
    train_fold(index)

print("load all models")
models = []

for index in range(CFG.n_folds):
    model = get_model()
    model.load_weights(f"model_{index}")
    models.append(model)

## Submission

In [ ]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0
def make_test_dataset(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds
def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
features = [f"f_{i}" for i in range(300)]
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df[features], test_df["investment_id"])
    sample_prediction_df['target'] = inference(models, ds)
    env.predict(sample_prediction_df) 

In [ ]:
submission=pd.read_csv("./submission.csv")
submission